# Homework 3 : Interactive Viz - PART 1

In this homework we will practice with interactive visualization, which is the key ingredient of many successful viz (especially when it comes to infographics). We will be work with the P3 database of the SNSF (Swiss National Science Foundation). 

In this homework, we are asked to build a colorpeth map which shows intuitively (i.e., use colors wisely) how much grant money goes to each Swiss canton.

In this file, we deal with two parts of the homework : 
- Data importing & cleaning
- Moving from university names to cantons

At the end, we obtain a DataFrame composed of two columns

## Data importing & cleaning

First, the raw data is downloaded directly from the website of the SNSF, and saved in the subdirectory 'data', under the name 'P3_GrantExport.csv'.

In [29]:
# basic imports
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import requests
import json
from bs4 import BeautifulSoup as BSoup
from keys import gkey

# reading the data
grant_data = pd.read_csv("data/P3_GrantExport.csv", sep=';', na_values="data not included in P3")
print(grant_data.shape)
grant_data.head()

(63969, 15)


,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.0,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.0,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.0,NaN
3,6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.0,NaN
4,7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.0,NaN


Each row of the data represents a scientific project. Informations about the university and the amount of money dedicated to the project are given in the columns 'University' and 'Approved amount'.

In this homework, we need to get the canton in which each university belongs, group the data by cantons, and compute the total amout of money given to each canton.

Let's start by cleaning the data first, to deal with wrong and missing values. 

In [30]:
# get only 'University' and 'Approved Amount'
grant_data = grant_data[['University','Approved Amount']]
grant_data.head()

,University,Approved Amount
0,Nicht zuteilbar - NA,11619.0
1,Université de Genève - GE,41022.0
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.0
3,Universität Basel - BS,52627.0
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.0


In [31]:
# drop NaNs of the 'University' column
uni_names = grant_data['University'].dropna().unique()
print(uni_names.shape[0])
grant_data = grant_data[grant_data['University'].isin(uni_names)]
print(grant_data.shape[0])

77
50988


- 12981 of elements of the university column are empty. After dropping missing values, 77 universities are left.

In [32]:
# Turn the type of 'Approved Amount' column to float.
grant_data['Approved Amount'] = grant_data['Approved Amount'].astype('float')
grant_data[np.isnan(grant_data['Approved Amount'])].shape[0]

110

- 110 elements from 63969 in the 'Approved Amount' column are missing. 

We decided to turn these elements to the mean of the approved amounts of the university.

In [33]:
# trasform the 'Approved Amount' column into the median approved amount of the university 
mean_amount = grant_data.groupby('University').transform('median')
print(mean_amount.shape[0])
print(grant_data.shape[0])

50988
50988


In [34]:
# fill NA's of the 'Approved Amount' of the correspondant values from the mean_amount dataframe
grant_data = grant_data.fillna(mean_amount)
grant_data.head()

,University,Approved Amount
0,Nicht zuteilbar - NA,11619.0
1,Université de Genève - GE,41022.0
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.0
3,Universität Basel - BS,52627.0
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.0


In [35]:
grant_data[np.isnan(grant_data['Approved Amount'])].shape[0]

0

At the end, we have no nan value in the 'Approved Amount' column.

## University name to Canton name

We need know to move, in a smart way, from the Universities names to the cantons names. 

In [8]:
# List of universities
uni_names

array(['Nicht zuteilbar - NA', 'Université de Genève - GE',
       'NPO (Biblioth., Museen, Verwalt.) - NPO', 'Universität Basel - BS',
       'Université de Fribourg - FR', 'Universität Zürich - ZH',
       'Université de Lausanne - LA', 'Universität Bern - BE',
       'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL',
       'Université de Neuchâtel - NE', 'ETH Zürich - ETHZ',
       'Inst. de Hautes Etudes Internat. et du Dév - IHEID',
       'Universität St. Gallen - SG', 'Weitere Institute - FINST',
       'Firmen/Privatwirtschaft - FP',
       'Pädagogische Hochschule Graubünden - PHGR', 'EPF Lausanne - EPFL',
       'Pädagogische Hochschule Zürich - PHZFH', 'Universität Luzern - LU',
       'Schweiz. Institut für Kunstwissenschaft - SIK-ISEA',
       'SUP della Svizzera italiana - SUPSI',
       'HES de Suisse occidentale - HES-SO',
       'Robert Walser-Stiftung Bern - RWS', 'Paul Scherrer Institut - PSI',
       'Pädagogische Hochschule St. Gallen - PHSG',
       'Eidg. Ans

We observed by simply using geocode of Google we only get 39% precision. 

We decided to use google Place API to get the long address of each university, and then use google Geocode API, to get the canton from the address. 

In [9]:
# function taking the name of the place and returning the long address of it
def getAddress(name):
  glink="https://maps.googleapis.com/maps/api/place/textsearch/json?query={}&key={}".format(name + " Switzerland", gkey)
  r = requests.get(glink)
  dic = json.loads(r.text)
  try:
    return dic['results'][0]['formatted_address']
  except:
    return ''

In [10]:
# Example
getAddress('EPFL')

'Route Cantonale, 1015 Lausanne, Switzerland'

We are going to apply this function on the list of all universities.

In [12]:
uni_addresses = [[a, getAddress(a)] for a in uni_names]
uni_addresses

[['Nicht zuteilbar - NA', ''],
 ['Université de Genève - GE', 'Geneva, Switzerland'],
 ['NPO (Biblioth., Museen, Verwalt.) - NPO', ''],
 ['Universität Basel - BS', 'Petersplatz 1, 4003 Basel, Switzerland'],
 ['Université de Fribourg - FR',
  "Avenue de l'Europe 20, 1700 Fribourg, Switzerland"],
 ['Universität Zürich - ZH', 'Rämistrasse 71, 8006 Zürich, Switzerland'],
 ['Université de Lausanne - LA', '1015 Lausanne, Switzerland'],
 ['Universität Bern - BE', 'Hochschulstrasse 6, 3012 Bern, Switzerland'],
 ['Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL',
  'Zürcherstrasse 111, 8903 Birmensdorf ZH, Switzerland'],
 ['Université de Neuchâtel - NE',
  'Av. du 1er-Mars 26, 2000 Neuchâtel, Switzerland'],
 ['ETH Zürich - ETHZ', 'Rämistrasse 101, 8092 Zürich, Switzerland'],
 ['Inst. de Hautes Etudes Internat. et du Dév - IHEID',
  'Maison de la Paix, Chemin Eugène-Rigot 2, 1202 Genève, Switzerland'],
 ['Universität St. Gallen - SG',
  'Dufourstrasse 50, 9000 St. Gallen, Switzerland'],
 ['We

Now we are going to save the results in a json file for later use.

In [13]:
with open('data/uniaddresses.json', 'w') as outfile:
    json.dump(uni_addresses, outfile)

Let's analyse the result.

In [14]:
# Filter the universities for which we have no result
uni_no_address = [a for a in uni_addresses if a[1] == ""]

In [15]:
# Computing the accuracy
(len(uni_addresses)-len(uni_no_address))*100/len(uni_addresses)

80.51948051948052

We have reached an accuracy of 80.52 %. We will need add some of the missing cantons manually to reach the required accuracy (95%).

In [16]:
uni_no_address

[['Nicht zuteilbar - NA', ''],
 ['NPO (Biblioth., Museen, Verwalt.) - NPO', ''],
 ['Weitere Institute - FINST', ''],
 ['Firmen/Privatwirtschaft - FP', ''],
 ['Robert Walser-Stiftung Bern - RWS', ''],
 ['Weitere Spitäler - ASPIT', ''],
 ['AO Research Institute - AORI', ''],
 ['Forschungsanstalten Agroscope - AGS', ''],
 ['Zürcher Fachhochschule (ohne PH) - ZFH', ''],
 ['Swiss Institute of Bioinformatics - SIB', ''],
 ['Forschungskommission SAGW', ''],
 ['Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP', ''],
 ['Schweizer Kompetenzzentrum Sozialwissensch. - FORS', ''],
 ['Pädagogische Hochschule Wallis - PHVS', ''],
 ['Staatsunabh. Theologische Hochschule Basel - STHB', '']]

Let's first get the cantons of the adresses we have using Google Geocode API.

In [17]:
geocodeLink = "https://maps.googleapis.com/maps/api/geocode/json?sensor=true"
from time import sleep

# Get the resulting Json file from the request, and put it in a json dictionary
def getGeocodeJsonDictionary(name_university):
    long_name = name_university.split(' - ')[0]
    link =  u"{}&address=[{}]".format(geocodeLink, long_name, gkey)
    sleep(0.2) # a workaround for the rate-limit of Google geocode API
    r = requests.get(link)
    dic = json.loads(r.text)
    return dic

# Get the canton from the administrative_area_level_1 field in the dictionary
def getCantonFromGeocodeDictionary(dic):
    a = dic['results'][0]['address_components']
    canton=[x['short_name'] for x in a if x['types'][0]=='administrative_area_level_1'][0]   
    return canton

# Get the canton by composing the two previous functions
def getCanton(name_university):
    return getCantonFromGeocodeDictionary(getGeocodeJsonDictionary(name_university))

We will apply the function getCanton to the adresses we have in uni_address.

In [18]:
uni_cantons = [[a[0], getCanton(a[1])] for a in uni_addresses if a[1] != ""]
uni_cantons

[['Université de Genève - GE', 'GE'],
 ['Universität Basel - BS', 'BS'],
 ['Université de Fribourg - FR', 'FR'],
 ['Universität Zürich - ZH', 'ZH'],
 ['Université de Lausanne - LA', 'VD'],
 ['Universität Bern - BE', 'BE'],
 ['Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL', 'ZH'],
 ['Université de Neuchâtel - NE', 'NE'],
 ['ETH Zürich - ETHZ', 'ZH'],
 ['Inst. de Hautes Etudes Internat. et du Dév - IHEID', 'Genève'],
 ['Universität St. Gallen - SG', 'SG'],
 ['Pädagogische Hochschule Graubünden - PHGR', 'GR'],
 ['EPF Lausanne - EPFL', 'VD'],
 ['Pädagogische Hochschule Zürich - PHZFH', 'ZH'],
 ['Universität Luzern - LU', 'LU'],
 ['Schweiz. Institut für Kunstwissenschaft - SIK-ISEA', 'ZH'],
 ['SUP della Svizzera italiana - SUPSI', 'TI'],
 ['HES de Suisse occidentale - HES-SO', 'JU'],
 ['Paul Scherrer Institut - PSI', 'AG'],
 ['Pädagogische Hochschule St. Gallen - PHSG', 'SG'],
 ['Eidg. Anstalt für Wasserversorgung - EAWAG', 'ZH'],
 ['Eidg. Material und Prüfungsanstalt - EMPA', 'SG'],
 

We possess now a list of universities and the cantons in which they are localized.
We will save the results in json files for later use.

In [19]:
with open('data/unicantons-auto.json', 'w') as outfile:
    json.dump(uni_cantons, outfile)
    
with open('data/unicantons-notspecified.json', 'w') as outfile:
    json.dump(uni_no_address, outfile)

The 'unicantons-auto.json' file contains the cantons generated by the API, and the 'unicantons-manual.json' contains those provided manually.

In [20]:
uni_cantons_auto=json.load(open('data/unicantons-auto.json'))
uni_cantons_man=json.load(open('data/unicantons-manual.json'))

Let's have a look on the manually added cantons.

Some of them are correcting previous results :
- for 'Inst. de Hautes Etudes Internat. et du Dév - IHEID' we have got Genève in the canton name, it should be 'GE'.
- 'Istituto Svizzero di Roma - ISR' is not in Switzerland, it should be Nan.

Some of them are Nas because they can not be assigned to a specific cantons : 
- Weitere Spit\xe4ler - ASPIT : other hospitals
- NPO (Biblioth., Museen, Verwalt.) - NPO : biblotheques, museems etc
- Firmen/Privatwirtschaft - FP : companies, private sectors
- etc
We decided to not consider these informations.

In [21]:
uni_cantons_man

[['Weitere Spitäler - ASPIT', ''],
 ['Forschungskommission SAGW', 'BE'],
 ['NPO (Biblioth., Museen, Verwalt.) - NPO', ''],
 ['Nicht zuteilbar - NA', ''],
 ['Firmen/Privatwirtschaft - FP', ''],
 ['Robert Walser-Stiftung Bern - RWS', 'BE'],
 ['Zürcher Fachhochschule (ohne PH) - ZFH', 'ZH'],
 ['Forschungsanstalten Agroscope - AGS', 'ZH'],
 ['Swiss Institute of Bioinformatics - SIB', 'VD'],
 ['Weitere Institute - FINST', 'BS'],
 ['AO Research Institute - AORI', 'GR'],
 ['Schweizer Kompetenzzentrum Sozialwissensch. - FORS', 'VD'],
 ['Pädagogische Hochschule Wallis - PHVS', 'VS'],
 ['Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP', 'TI'],
 ['Staatsunabh. Theologische Hochschule Basel - STHB', 'BS'],
 ['Inst. de Hautes Etudes Internat. et du Dév - IHEID', 'GE'],
 ['Istituto Svizzero di Roma - ISR', '']]

We are now going to concatenate the two files into one unique variable combining all the informations.

In [22]:
uni_cantons_all=uni_cantons_man+uni_cantons_auto
uni_cantons_all

[['Weitere Spitäler - ASPIT', ''],
 ['Forschungskommission SAGW', 'BE'],
 ['NPO (Biblioth., Museen, Verwalt.) - NPO', ''],
 ['Nicht zuteilbar - NA', ''],
 ['Firmen/Privatwirtschaft - FP', ''],
 ['Robert Walser-Stiftung Bern - RWS', 'BE'],
 ['Zürcher Fachhochschule (ohne PH) - ZFH', 'ZH'],
 ['Forschungsanstalten Agroscope - AGS', 'ZH'],
 ['Swiss Institute of Bioinformatics - SIB', 'VD'],
 ['Weitere Institute - FINST', 'BS'],
 ['AO Research Institute - AORI', 'GR'],
 ['Schweizer Kompetenzzentrum Sozialwissensch. - FORS', 'VD'],
 ['Pädagogische Hochschule Wallis - PHVS', 'VS'],
 ['Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP', 'TI'],
 ['Staatsunabh. Theologische Hochschule Basel - STHB', 'BS'],
 ['Inst. de Hautes Etudes Internat. et du Dév - IHEID', 'GE'],
 ['Istituto Svizzero di Roma - ISR', ''],
 ['Université de Genève - GE', 'GE'],
 ['Universität Basel - BS', 'BS'],
 ['Université de Fribourg - FR', 'FR'],
 ['Universität Zürich - ZH', 'ZH'],
 ['Université de Lausanne - LA', 'VD'],


We will put the results into a dataframe composed of two columns : 'University' and 'Canton'.

In [25]:
df_uni_cantons=pd.DataFrame(uni_cantons_all, columns=['University', 'Canton'])
# fixing a unicode issue (special characters)
df_uni_cantons["University"]=[a.encode("utf-8") for a in df_uni_cantons["University"]]
# overriding manually entered universities
df_uni_cantons=df_uni_cantons.drop_duplicates(subset=['University'],keep='first')
# remove universities without canton info
df_uni_cantons=df_uni_cantons[df_uni_cantons['Canton']!=""]
df_uni_cantons

,University,Canton
1,b'Forschungskommission SAGW',BE
5,b'Robert Walser-Stiftung Bern - RWS',BE
6,b'Z\xc3\xbcrcher Fachhochschule (ohne PH) - ZFH',ZH
7,b'Forschungsanstalten Agroscope - AGS',ZH
8,b'Swiss Institute of Bioinformatics - SIB',VD
9,b'Weitere Institute - FINST',BS
10,b'AO Research Institute - AORI',GR
11,b'Schweizer Kompetenzzentrum Sozialwissensch. ...,VD
12,b'P\xc3\xa4dagogische Hochschule Wallis - PHVS',VS
13,b'P\xc3\xa4dag. Hochschule Tessin (Teilschule ...,TI


We need to merge the canton information into the data.

In [36]:
# fixing a unicode issue (special characters)
grant_data["University"]=[a.encode("utf-8") for a in grant_data["University"]]

In [37]:
grant_data.head()

,University,Approved Amount
0,b'Nicht zuteilbar - NA',11619.0
1,b'Universit\xc3\xa9 de Gen\xc3\xa8ve - GE',41022.0
2,"b'NPO (Biblioth., Museen, Verwalt.) - NPO'",79732.0
3,b'Universit\xc3\xa4t Basel - BS',52627.0
4,"b'NPO (Biblioth., Museen, Verwalt.) - NPO'",120042.0


In [38]:
df_canton_all_grant=pd.merge(grant_data, df_uni_cantons)[["Canton", "Approved Amount"]]
df_canton_all_grant.head()

,Canton,Approved Amount
0,GE,41022.0
1,GE,360000.0
2,GE,116991.0
3,GE,149485.0
4,GE,164602.0


The next step is to sum the approved amounts of each canton.

In [39]:
# group the data & sum approved amounts
df_canton_grant=df_canton_all_grant.groupby('Canton').sum()
# sort values
df_canton_grant_sorted=df_canton_grant.sort_values(['Approved Amount'], ascending=False)
df_canton_grant_sorted

,Approved Amount
Canton,
ZH,3.680964e+09
VD,2.417983e+09
GE,1.880155e+09
BE,1.557800e+09
BS,1.403558e+09
FR,4.592306e+08
NE,4.031020e+08
SG,1.497686e+08
AG,1.228849e+08


We are going to save the results in a CSV file, to plot them in a colorpeth map.

In [40]:
df_canton_grant.to_csv('data/canton_amount.csv')